# Embedded Python with JavaCPP

> Installation instructions and small experiments with JavaCPP embedded Python

- toc: false
- branch: master
- badges: true
- comments: true
- author: Stephan Saalfeld
- categories: [javacpp, java, python]

We explored how to interact between Java and Python using [JavaCPP embeded Python](https://github.com/bytedeco/javacpp-embedded-python).  It turns out that updating a few versions here and there is necessary to work with the latest distribution.  Here are the instructions:

## Install JavaCPP embedded Python

First clone the repository:
```
git clone https://github.com/bytedeco/javacpp-embedded-python.git
```

Then, in its `pom.xml` file, bump the numpy dependencies to:
```xml
<dependency>
    <groupId>org.bytedeco</groupId>
    <artifactId>numpy</artifactId>
    <version>1.22.2-1.5.7</version>
</dependency>

<dependency>
    <groupId>org.bytedeco</groupId>
    <artifactId>numpy-platform</artifactId>
    <version>1.22.2-1.5.7</version>
    <scope>test</scope>
</dependency>
```

Then build and locally install the project with maven.  I had to skip Javadoc generation because my Javadoc is misconfigured:
```
mvn -Dmaven.javadoc.skip=true clean install
```

Done.

## Experiments

Now let's test this in a notebook with the [IJava JShell kernel](https://github.com/SpencerPark/IJava).  If you haven't done this yet, build and install the most recent kernel following the instructions on the [ImgLib2 blog](https://imglib.github.io/imglib2-blog/jupyter/ijava/jshell/java/kernel/2022/06/05/setup-ijava-jupyter-kernel.html).

We can now link JavaCPP embedded Python as a dependency:

In [1]:
%maven org.bytedeco:javacpp:1.5.7
%maven org.bytedeco:cpython-platform:3.10.2-1.5.7
%maven org.bytedeco:numpy-platform:1.22.2-1.5.7
%maven org.bytedeco:javacpp-embedded-python:1.0.0-SNAPSHOT

Let's test it:

In [2]:
import org.bytedeco.embeddedpython.*;

Python.put("a", 1);
Python.exec("b = a + 2");
long b = Python.get("b");
b;

3

We can also install some packages and find out where they go:

In [3]:
Pip.install("pandas");
Python.exec("import pandas");
Python.eval("str(pandas)");

<module 'pandas' from '/home/saalfeld/.javacpp/cache/cpython-3.10.2-1.5.7-linux-x86_64.jar/org/bytedeco/cpython/linux-x86_64/lib/python3.10/site-packages/pandas/__init__.py'>

Here is a function that forwards to a Python function by name:

In [4]:
public Object processWithPython(
        final String functionName,
        final Object... parameters) {
        
    Python.put("args", parameters);
        
    return Python.eval(functionName + "(*args)");
}

processWithPython("len", new int[]{1, 2, 3});

3

In [5]:
processWithPython("sum", new int[]{1, 2, 3});

6

In [6]:
import java.util.function.Function;

public <I, O> Function<I, O> createPythonFunction(
        final String functionName) {
        
        return (parameters) -> {
        
            Python.put("args", parameters);
            return Python.eval(functionName + "(args)");
        };
}

var len = createPythonFunction("len");
len.apply(new int[]{1, 2, 3});

3

In [7]:
var sum = createPythonFunction("sum");
sum.apply(new int[]{1, 2, 3});

6

To be continued...